In [26]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import time

In [27]:
csv_file = '../datasets/chicago_trimmed.csv'
df = pd.read_csv(csv_file)
org_df = df.copy()

In [50]:
# finding one row in a dataframe
org_df.iloc[100004]

date                    2009-03-08 11:40:00+00:00
primary_type                    CRIMINAL TRESPASS
description                          TO RESIDENCE
location_description                    RESIDENCE
arrest                                       True
latitude                                 41.80908
longitude                              -87.633906
Name: 100004, dtype: object

In [38]:
df.head()

,date,primary_type,description,location_description,arrest,latitude,longitude
0,2010-10-03 11:30:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926
1,2005-10-31 03:55:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.751061,-87.569876
2,2003-10-07 02:30:00+00:00,BATTERY,SIMPLE,APARTMENT,True,41.751215,-87.571098
3,2009-01-22 02:30:00+00:00,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,41.733650,-87.557845
4,2020-07-03 12:15:00+00:00,BATTERY,SIMPLE,RESIDENCE,False,41.749243,-87.549214


In [39]:
# take a sample for testing
df = df[:100010]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100010 entries, 0 to 100009
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   date                  100010 non-null  object 
 1   primary_type          100010 non-null  object 
 2   description           100010 non-null  object 
 3   location_description  100010 non-null  object 
 4   arrest                100010 non-null  bool   
 5   latitude              100010 non-null  float64
 6   longitude             100010 non-null  float64
dtypes: bool(1), float64(2), object(4)
memory usage: 4.7+ MB


In [40]:
df['primary_type'][:60000].value_counts()

THEFT                                12494
BATTERY                              10822
CRIMINAL DAMAGE                       7504
NARCOTICS                             4688
ASSAULT                               4168
OTHER OFFENSE                         3561
BURGLARY                              3336
DECEPTIVE PRACTICE                    2977
MOTOR VEHICLE THEFT                   2811
ROBBERY                               2179
CRIMINAL TRESPASS                     1830
WEAPONS VIOLATION                      781
OFFENSE INVOLVING CHILDREN             524
PUBLIC PEACE VIOLATION                 462
PROSTITUTION                           391
SEX OFFENSE                            286
CRIM SEXUAL ASSAULT                    285
INTERFERENCE WITH PUBLIC OFFICER       154
LIQUOR LAW VIOLATION                   139
ARSON                                  136
GAMBLING                               108
HOMICIDE                               107
KIDNAPPING                             104
INTIMIDATIO

In [41]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df['date_year'] = df['date'].dt.year
df['date_month'] = df['date'].dt.month
df['date_day'] = df['date'].dt.day
df['date_dayofweek'] = df['date'].dt.dayofweek
df['date_hour'] = df['date'].dt.hour
df['date_minute'] = df['date'].dt.minute

df = df.drop(['date'], axis=1)

In [42]:
df.head()

,primary_type,description,location_description,arrest,latitude,longitude,date_year,date_month,date_day,date_dayofweek,date_hour,date_minute
0,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926,2010,10,3,6,11,30
1,BATTERY,SIMPLE,SIDEWALK,False,41.751061,-87.569876,2005,10,31,0,3,55
2,BATTERY,SIMPLE,APARTMENT,True,41.751215,-87.571098,2003,10,7,1,2,30
3,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,41.733650,-87.557845,2009,1,22,3,2,30
4,BATTERY,SIMPLE,RESIDENCE,False,41.749243,-87.549214,2020,7,3,4,12,15


In [43]:
len(df['location_description'].unique())

163

### Label encode the non-numerical features

In [8]:
%%perl -e0
label_encoder = preprocessing.LabelEncoder()
df['primary_type'] = label_encoder.fit_transform(df['primary_type'])
df['description'] = label_encoder.fit_transform(df['description'])
df['location_description'] = label_encoder.fit_transform(df['location_description'])

### One hot encode the label encoded features

In [44]:
one_hot_encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')

prim_type_df = pd.DataFrame(one_hot_encoder.fit_transform(df[['primary_type']]).toarray())
prim_type_df.columns = one_hot_encoder.get_feature_names_out(['primary_type'])

description_df = pd.DataFrame(one_hot_encoder.fit_transform(df[['description']]).toarray())
description_df.columns = one_hot_encoder.get_feature_names_out(['description'])

location_description_df = pd.DataFrame(one_hot_encoder.fit_transform(df[['location_description']]).toarray())
location_description_df.columns = one_hot_encoder.get_feature_names_out(['location_description'])

df = df.join(prim_type_df)
df = df.join(description_df)
df = df.join(location_description_df)

In [49]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100010 entries, 0 to 100009
Columns: 593 entries, arrest to location_description_YARD
dtypes: bool(1), float64(586), int64(6)
memory usage: 451.8 MB


(100010, 593)

In [46]:
df.head()

,primary_type,description,location_description,arrest,latitude,longitude,date_year,date_month,date_day,date_dayofweek,...,location_description_VACANT LOT/LAND,location_description_VEHICLE - COMMERCIAL,location_description_VEHICLE - DELIVERY TRUCK,location_description_VEHICLE - OTHER RIDE SERVICE,"location_description_VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)","location_description_VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)",location_description_VEHICLE NON-COMMERCIAL,location_description_VEHICLE-COMMERCIAL,location_description_WAREHOUSE,location_description_YARD
0,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926,2010,10,3,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BATTERY,SIMPLE,SIDEWALK,False,41.751061,-87.569876,2005,10,31,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BATTERY,SIMPLE,APARTMENT,True,41.751215,-87.571098,2003,10,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,41.733650,-87.557845,2009,1,22,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BATTERY,SIMPLE,RESIDENCE,False,41.749243,-87.549214,2020,7,3,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df = df.drop(['primary_type', 'description', 'location_description'], axis=1)
df.head()

,arrest,latitude,longitude,date_year,date_month,date_day,date_dayofweek,date_hour,date_minute,primary_type_ARSON,...,location_description_VACANT LOT/LAND,location_description_VEHICLE - COMMERCIAL,location_description_VEHICLE - DELIVERY TRUCK,location_description_VEHICLE - OTHER RIDE SERVICE,"location_description_VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)","location_description_VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)",location_description_VEHICLE NON-COMMERCIAL,location_description_VEHICLE-COMMERCIAL,location_description_WAREHOUSE,location_description_YARD
0,False,41.745809,-87.547926,2010,10,3,6,11,30,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,41.751061,-87.569876,2005,10,31,0,3,55,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,41.751215,-87.571098,2003,10,7,1,2,30,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,41.733650,-87.557845,2009,1,22,3,2,30,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,41.749243,-87.549214,2020,7,3,4,12,15,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cast numerical datatype to ratio data

In [13]:
# should coordinates be ratio?

### Create feature and target

In [14]:
X = df.drop(['arrest'], axis=1)
y = df['arrest'].astype(int) # convert boolean to int

In [15]:
X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:100000]
y_test = y[60000:100000]
X_val = X[100000:]
y_val = y[100000:]

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100010 entries, 0 to 100009
Columns: 592 entries, latitude to location_description_YARD
dtypes: float64(586), int64(6)
memory usage: 451.7 MB


In [19]:
classifier_names = [
    #"Random Forest (n_estimators=100, n_jobs=-1)", 
    #"Random Forest (n_estimators=500, n_jobs=-1)", 
    #"Random Forest (n_estimators=1000, n_jobs=-1)", 
    "Neural Net (100, 100), adam",
    "Neural Net (100, 100), sgd", 
    "Neural Net (100, 100, 100), adam", 
    "Neural Net (100, 50, 25), adam", # hidden layers
]

classifiers = [
    #RandomForestClassifier(n_estimators=100, n_jobs=-1),
    #RandomForestClassifier(n_estimators=500, n_jobs=-1),
    #RandomForestClassifier(n_estimators=1000, n_jobs=-1),
    MLPClassifier(hidden_layer_sizes=(100, 100)),
    MLPClassifier(hidden_layer_sizes=(100, 100), solver=('sgd')),
    MLPClassifier(hidden_layer_sizes=(100, 100, 100)),
    MLPClassifier(hidden_layer_sizes=(592, 296, 148)),
]

for clf, clf_name in zip(classifiers, classifier_names):
    print(f"** {clf_name}")
    t0 = time.time()
    clf.fit(X_train, y_train)
    t1 = time.time()
    print(f"\tTraining time:\t\t{t1-t0:3.3f}")
    score_train = clf.score(X_train[0:10000], y_train[0:10000])
    t2 = time.time()
    print(f"\tPrediction time(train):\t{t2-t1:3.3f}")
    score_test = clf.score(X_test, y_test)
    t3 = time.time()
    print(f"\tPrediction time(test):\t{t3-t2:3.3f}")
    print(f"\tScore Train: {score_train:.3f}\tScore Test: {score_test:.3f}")

** Neural Net (100, 100), adam
	Training time:		261.515
	Prediction time(train):	0.105
	Prediction time(test):	0.401
	Score Train: 0.849	Score Test: 0.884
** Neural Net (100, 100), sgd
	Training time:		19.101
	Prediction time(train):	0.082
	Prediction time(test):	0.290
	Score Train: 0.729	Score Test: 0.739
** Neural Net (100, 100, 100), adam
	Training time:		157.047
	Prediction time(train):	0.141
	Prediction time(test):	0.583
	Score Train: 0.846	Score Test: 0.879
** Neural Net (592, 296, 148), adam
	Training time:		1109.105
	Prediction time(train):	0.369
	Prediction time(test):	1.396
	Score Train: 0.848	Score Test: 0.883


## Results from last run 
```
** Random Forest (n_estimators=100, n_jobs=-1)
	Training time:		8.515
	Prediction time(train):	0.175
	Prediction time(test):	0.574
	Score Train: 1.000	Score Test: 0.886
** Random Forest (n_estimators=500, n_jobs=-1)
	Training time:		39.427
	Prediction time(train):	0.667
	Prediction time(test):	2.075
	Score Train: 1.000	Score Test: 0.886
** Random Forest (n_estimators=1000, n_jobs=-1)
	Training time:		73.081
	Prediction time(train):	1.046
	Prediction time(test):	3.565
	Score Train: 1.000	Score Test: 0.886
** Neural Net (100, 100), adam
	Training time:		261.515
	Prediction time(train):	0.105
	Prediction time(test):	0.401
	Score Train: 0.849	Score Test: 0.884
** Neural Net (100, 100), sgd
	Training time:		19.101
	Prediction time(train):	0.082
	Prediction time(test):	0.290
	Score Train: 0.729	Score Test: 0.739
** Neural Net (100, 100, 100), adam
	Training time:		157.047
	Prediction time(train):	0.141
	Prediction time(test):	0.583
	Score Train: 0.846	Score Test: 0.879
** Neural Net (592, 296, 148), adam
	Training time:		1109.105
	Prediction time(train):	0.369
	Prediction time(test):	1.396
	Score Train: 0.848	Score Test: 0.883
    ```

In [18]:
%%perl -e0
model.predict(X_val)
org_df.iloc[100004]